<a href="https://colab.research.google.com/github/CeciliaMarson/CMLS-Homework1/blob/master/Braian_HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 - Computer Music (Valence and Arousal Classification)

In [0]:
import numpy as np
import librosa, librosa.display
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp

In [0]:
!wget --no-check-certificate -r "https://drive.google.com/uc?id=12MCAyfZxquIK-dkmOX3Mi9AC2SBSJ2bG&export=download" -O "MEMD_audio.zip" 

In [0]:
!unzip MEMD_audio.zip

In [0]:

ipd.Audio('/content/1984.mp3') #NINE INCH NAILS!!!!!! <3

# Feature definition

### Define parameters and variables for feature computation

In [0]:
from sklearn.model_selection import train_test_split
songs = '/content/'
all_songs = [f for f in os.listdir(songs) if f.endswith('.mp3')]
songs_train, songs_test = train_test_split(all_songs, test_size=0.99)


In [0]:
  for index, f in enumerate(songs_train):
    audio, fs = librosa.load(os.path.join(songs, f), sr=None)
    roll = librosa.feature.spectral_rolloff(audio, sr=fs, S=None, n_fft=1024, hop_length=512, freq=None, roll_percent=0.75)
    zcr =  librosa.feature.zero_crossing_rate(audio, frame_length=1024, hop_length=512, center=True)


In [0]:
roll.T

In [0]:
np.savetxt("1985_roll_csv.csv", roll.T, delimiter=",")

### Compute training features